Suumoからデータ抽出

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# SuumoのURLのベース部分
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=20.0&et=9999999&md=05&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"


# 物件情報を格納するリスト
properties = []

# セッションを開始
session = requests.Session()

# 最初のページからスタート
page = 1

while True:
    # パラメータにページ数を追加
    params = {'page': page}

    # リクエストを送信
    response = session.get(base_url, params=params)
    response.encoding = response.apparent_encoding

    # HTMLコンテンツを解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # 物件情報の抽出
    listings = soup.find_all('div', class_='cassetteitem')
    if not listings:
        break  # リストが空なら終了

    for listing in listings:
        property_name = listing.find('div', class_='cassetteitem_content-title').text.strip()
        address = listing.find('li', class_='cassetteitem_detail-col1').text.strip()
        layout = listing.find('span', class_='cassetteitem_madori').text.strip()
        rent = listing.find('span', class_='cassetteitem_other-emphasis').text.strip()
        floor = listing.find('li', class_='cassetteitem_detail-col3').text.strip()
        access = listing.find('div', class_='cassetteitem_detail-text').text.strip()

        properties.append({
            '物件名': property_name,
            '住所': address,
            '間取り': layout,
            '家賃': rent,
            '階数': floor,
            '最寄りまでの時間': access
        })

    # 次のページへ
    page += 1

        # サーバーへの負荷を考慮して少し待機
    time.sleep(1)


# 最終的な物件情報のリストを表示
for property in properties:
    print(property)

# 'properties'リストをデータフレームに変換
properties_df = pd.DataFrame(properties)

# データフレームの表示
print(properties_df)


/Users/maedaryo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'物件名': 'クレイシアＩＤＺ本所吾妻橋', '住所': '東京都墨田区本所４', '間取り': '2K', '家賃': '10.2万円', '階数': '築1年\n9階建', '最寄りまでの時間': '都営浅草線/本所吾妻橋駅 歩7分'}
{'物件名': 'ＪＲ総武線 錦糸町駅 15階建 築2年', '住所': '東京都墨田区江東橋２', '間取り': '2K', '家賃': '10.6万円', '階数': '築2年\n15階建', '最寄りまでの時間': 'ＪＲ総武線/錦糸町駅 歩7分'}
{'物件名': 'ライフＫIII', '住所': '東京都荒川区西尾久３', '間取り': '2K', '家賃': '6.5万円', '階数': '築35年\n2階建', '最寄りまでの時間': '都電荒川線/小台駅 歩5分'}
{'物件名': 'アークマーク中野坂上', '住所': '東京都中野区東中野１', '間取り': '2K', '家賃': '11万円', '階数': '築4年\n5階建', '最寄りまでの時間': 'ＪＲ総武線/東中野駅 歩7分'}
{'物件名': 'ハーモニーレジデンス中野本町パークサイド', '住所': '東京都中野区本町５', '間取り': '2K', '家賃': '10.75万円', '階数': '築3年\n4階建', '最寄りまでの時間': '東京メトロ丸ノ内線/中野新橋駅 歩6分'}
{'物件名': '東京メトロ丸ノ内線 中野新橋駅 4階建 築3年', '住所': '東京都中野区本町５', '間取り': '2K', '家賃': '10.75万円', '階数': '築3年\n4階建', '最寄りまでの時間': '東京メトロ丸ノ内線/中野新橋駅 歩6分'}
{'物件名': 'ラフィスタ方南町', '住所': '東京都中野区南台５', '間取り': '2K', '家賃': '9.95万円', '階数': '築3年\n5階建', '最寄りまでの時間': '東京メトロ丸ノ内線/方南町駅 歩8分'}
{'物件名': '東京メトロ千代田線 北綾瀬駅 6階建 築4年', '住所': '東京都足立区加平１', '間取り': '2K', '家賃': '6.5万円', '階数': '築4年\n6階建', '最寄りまでの時間': '東京メトロ千代田線/北綾瀬

In [3]:
properties# スクレイピングしたデータの件数を表示
data_count = len(properties)
print(f"抽出された物件の件数: {data_count}")


抽出された物件の件数: 5441


スプレッドシートで出力。

In [34]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from dotenv import load_dotenv
load_dotenv()
import os


SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE=os.getenv('SERVICE_ACCOUNT_FILE')
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)

gs = gspread.authorize(credentials)
SPREADSHEET_KEY = os.getenv("SPREADSHEET_KEY")
workbook = gs.open_by_key("SPREADSHEET_KEY")
worksheet = workbook.worksheet("DB")

ModuleNotFoundError: No module named 'gspread_dataframe'

In [23]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os


# APIキーのパスとスプレッドシートのキー
api_key_path = os.getenv("api_key_path")
spreadsheet_key = os.getenv("spreadsheet_key")

# スコープの設定
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# 認証
credentials = ServiceAccountCredentials.from_json_keyfile_name(api_key_path, scope)
client = gspread.authorize(credentials)

# スプレッドシートを開く
spreadsheet = client.open_by_key(spreadsheet_key)
worksheet = spreadsheet.get_worksheet(0)  # 最初のシートを選択

# データフレームをスプレッドシートに書き込む
worksheet.update([properties_df.columns.values.tolist()] + properties_df.values.tolist())


TypeError: expected str, bytes or os.PathLike object, not NoneType